In [9]:
import pandas as pd    
pd.set_option('mode.chained_assignment', None)
import numpy as np

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import make_scorer

from aux_fun import my_eval, my_grid_search_cv

import json

In [10]:
df = pd.read_csv('./data/train_imputed.csv')

In [11]:
y = df['NumberOfSales']

In [12]:
X = df.drop(df[['NumberOfSales','NumberOfCustomers', 'WindDirDegrees']], axis=1)

# Train-Test Split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
X_train = X_train.reset_index(drop =True)
y_train = y_train.reset_index(drop =True)

# Find the combination with best parameters

In [ ]:
#parameters
params_array = [
    {'loss' : 'ls', 'n_estimators' : 50},
     {'loss' : 'huber', 'n_estimators' : 100},
    {'loss' : 'ls', 'max_depth' : 3, 'n_estimators' : 100},
    {'n_estimators' : 150},    
]

In [ ]:
results = my_grid_search_cv(GradientBoostingRegressor(), params_array, X_train, y_train, n_folds=5)

In [ ]:
results

In [ ]:
results.to_csv('GradientBoostingRegressor.csv')

# Model Building

In [ ]:
best_params = results[results['Eval_test']==min(results['Eval_test'])]['Parameters'].reset_index(drop =True)[0]

In [ ]:
best_params

In [ ]:
GBR = GradientBoosting(**best_params)
GBR.fit(X_train, y_train)

In [ ]:
y_pred = GBR.predict(X_test)

In [ ]:
print(y_pred)

In [ ]:
print(y_test)

In [ ]:
evaluation = my_eval(X_test, y_test, y_pred)
evaluation

In [ ]:
r2 = r2_score(y_test, y_pred)
r2

# Creation of the submission 

In [ ]:
model_result={'method': 'GradientBoostingRegressor',
       'parameters':len(params_array),
       'evaluation': evaluation, 
       'r2':r2,
       'notes': 'None'
    }

In [ ]:
with open('results.json', 'a') as fp:
    json.dump(model_result, fp, indent=2)
    fp.write('\n')